In [1]:
import pandas as pd
import bs4
import json
import re
import tqdm

import scipy.stats as st
import numpy as np

import bokeh
from bokeh.plotting import show as show_interactive
from bokeh.plotting import output_file, output_notebook
from bokeh.layouts import column, row
from bokeh.models import CustomJS, TextInput, LassoSelectTool, Select, MultiSelect, ColorBar, Legend, LegendItem
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Button, HTMLTemplateFormatter
from bokeh.events import SelectionGeometry
from bokeh.transform import linear_cmap, jitter
from bokeh.models import FactorRange


In [2]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')
go_df = pd.read_csv('./go_annotations.csv')
kegg_df = pd.read_csv('./kegg_annotations.csv')
ec_df = pd.read_csv('./ec_annotations.csv')

In [3]:
go_df.head()

,GOs,GOs_description,GOs_definition,GOs_obsolete
0,-,-,-,-
1,GO:0000001,mitochondrion inheritance,"The distribution of mitochondria, including th...",False
2,GO:0000002,mitochondrial genome maintenance,The maintenance of the structure and integrity...,False
3,GO:0000003,reproduction,The production of new individuals that contain...,False
4,GO:0000011,vacuole inheritance,The distribution of vacuoles into daughter cel...,False


In [4]:
go_df['GOs_description'].loc[go_df['GOs'] == 'GO:0000001'].values[0]

'mitochondrion inheritance'

In [5]:
kegg_df.head()

,KEGG_ko,KEGG_ko_description
0,-,-
1,ko:K00002,alcohol dehydrogenase (NADP+)
2,ko:K00006,glycerol-3-phosphate dehydrogenase (NAD+)
3,ko:K00011,aldehyde reductase
4,ko:K00012,UDPglucose 6-dehydrogenase


In [6]:
ec_df.head()

,EC,EC_description
0,-,-
1,1.1.1.1,alcohol dehydrogenase
2,1.1.1.105,all-trans-retinol dehydrogenase (NAD(+))
3,1.1.1.141,15-hydroxyprostaglandin dehydrogenase (NAD(+))
4,1.1.1.178,3-hydroxy-2-methylbutyryl-CoA dehydrogenase


In [7]:
# As of 2020 https://www.ncbi.nlm.nih.gov/research/cog/
COG_dict = {
    "A" : "RNA processing and modification",
    "B" : "Chromatin structure and dynamics",
    "C" : "Energy production and conversion",
    "D" : "Cell cycle control, cell division, chromosome partitioning",
    "E" : "Amino acid transport and metabolism",
    "F" : "Nucleotide transport and metabolism",
    "G" : "Carbohydrate transport and metabolism",
    "H" : "Coenzyme transport and metabolism",
    "I" : "Lipid transport and metabolism",
    "J" : "Translation, ribosomal structure and biogenesis",
    "K" : "Transcription",
    "L" : "Replication, recombination, and repair",
    "M" : "Cell wall/membrane/envelope biogenesis",
    "N" : "Cell motility",
    "O" : "Posttranslational modification, protein turnover, chaperones",
    "P" : "Inorganic ion transport and metabolism",
    "Q" : "Secondary metabolites biosynthesis, transport and catabolism",
    "T" : "Signal transduction mechanisms",
    "U" : "Intracellular trafficking, secretion, and vesicular transport",
    "V" : "Defense mechanisms",
    "W" : "Extracellular structures",
    "X" : "Mobilome: prophages, transposons",
    "Y" : "Nuclear structure",
    "Z" : "Cytoskeleton",
    "R" : "General function prediction only",
    "S" : "Function unknown",
}

In [8]:
def term_count_dict_from_annotation_df(annot_df, term_column):
    
    column = annot_df[term_column].values
    
    funct_terms = []
    for entry in column:
        if entry != '-':
            if term_column == 'COG_category':
                # terms = [f'{e}: {COG_dict[e]}' for e in entry]
                terms = list(entry)
            else:
                terms = entry.split(',')
            for t in terms:
                funct_terms.append(t)

#     len(terms)
    
    term_count_dict = {}
    
    for t in funct_terms:
        count = term_count_dict.get(t, 0)
        count += 1
        term_count_dict[t] = count
        
    return term_count_dict

In [9]:
def enrichment_analysis(module, leiden_label_df, phases, background_annotation, term_column):
    
    module_ttids = leiden_label_df.loc[leiden_label_df[f'leiden_label_{phases}'] == module]['TTHERM_ID'].values
    
    module_annotation = background_annotation.loc[background_annotation['TTHERM_ID'].isin(module_ttids)]
    
    background_term_dict = term_count_dict_from_annotation_df(background_annotation, term_column)
    module_term_dict = term_count_dict_from_annotation_df(module_annotation, term_column)
    
    bs = []
    ps = []
    folds = []
    terms = []
    
    for t, module_count in module_term_dict.items():
        
        background_count = background_term_dict[t]
        module_size = len(module_annotation)
        background_size = len(background_annotation)
        
        standard_contingency_table = [
                                [module_count, background_count - module_count], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        # The -1 and +1 make this more conservative (see explanation from the DAVID database: 
        # https://david.ncifcrf.gov/helps/functional_annotation.html#geneenrich)
        conservative_contingency_table = [
                                [module_count - 1, background_count - module_count + 1], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        
        odds, p_standard = st.fisher_exact(standard_contingency_table, 'greater')
        odds, p_conservative = st.fisher_exact(conservative_contingency_table, 'greater')
        
        p_reasonable = np.mean([p_standard, p_conservative])
        
        bonferroni  = p_reasonable * len(module_term_dict)

        fold_enrichment = (module_count/module_size) / (background_count/background_size)

        if bonferroni <= 0.05:
            
            ps.append(p_reasonable)
            bs.append(bonferroni)
            folds.append(fold_enrichment)
            terms.append(t)
            
#         else:
#             ps.append('')
#             bs.append('')
#             folds.append('')
#             terms.append('')
            
    return ps, bs, folds, terms
            
            

In [10]:
def get_GO_info(go_term):
    
    name = go_df['GOs_description'].loc[go_df['GOs'] == go_term].values[0]
    
    definition = go_df['GOs_definition'].loc[go_df['GOs'] == go_term].values[0]
    
    obsolete = go_df['GOs_obsolete'].loc[go_df['GOs'] == go_term].values[0]
    
    return name, definition, obsolete

In [11]:
get_GO_info('GO:0000001')

('mitochondrion inheritance',
 'The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton.',
 'False')

In [12]:
def get_KEGG_info(term):
    return kegg_df['KEGG_ko_description'].loc[kegg_df['KEGG_ko'] == term].values[0]

In [13]:
get_KEGG_info('ko:K00002')

'alcohol dehydrogenase (NADP+)'

In [14]:
def get_EC_info(term):
    return ec_df['EC_description'].loc[ec_df['EC'] == term].values[0]

In [15]:
get_EC_info('1.1.1.141')

'15-hydroxyprostaglandin dehydrogenase (NAD(+))'

In [16]:
def write_enrichment(lldf, phases, out_prefix, term_column):
    
    with open(f'../enrichment/{out_prefix}_enrichment.txt', 'w') as fl:
    
        for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):

            # print(f'Module {m}')
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, complete_annot, term_column)

            fl.write(f'Module {m}\n')
            for b, fold, t in zip(bs, folds, terms):
                
                if term_column == 'COG_category':
                    
                    definition = COG_dict[t]
                    t = f'{t}: {definition}'
                
                elif term_column == 'GOs':
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
                    else:
                        t = f'{name.capitalize()} ({t}): {definition}'
                        
                elif term_column.split('_')[0] == 'KEGG':
                    name = get_KEGG_info(t)
                    
                    t = f'{t}: {name}'
                    
                elif term_column == 'EC':
                    
                    name = get_EC_info(t)
                    t = f'{t}: {name}'
                    
                        
                # print(f"{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n")

                fl.write(f'{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n')

In [17]:
def get_enrichment_df(lldf, phases, background_annotation, term_columns=['COG_category', 'GOs', 'KEGG_ko', 'EC'], outfile=None):
    
    module_dfs = []
    
    for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):
        
        term_dfs = []

        for tc in term_columns:
        
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, background_annotation, tc)

            # fl.write(f'Module {m}\n')
            
            info = []

            if tc == 'GOs':

                for t in terms:
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        info.append(f'{name.capitalize()}: {definition} (obsolete)')
                    else:
                        info.append(f'{name.capitalize()}: {definition}')
                        
            elif tc == 'COG_category':
                for t in terms:
                    info.append(COG_dict[t])
                                
            elif tc == 'KEGG_ko':
                for t in terms:
                    info.append(get_KEGG_info(t))
                    
            elif tc == 'EC':
                for t in terms:
                    info.append(get_EC_info(t))
                    
            term_df = pd.DataFrame({'module': [m]*len(terms),
                                    'term': terms,
                                    'info': info,
                                    'fold_change': folds,
                                    'bonferroni': bs})
            
            term_dfs.append(term_df)
            
        module_df = pd.concat(term_dfs)
        
        module_dfs.append(module_df)
        
    all_enrichment_df = pd.concat(module_dfs)
    
    if outfile:
        all_enrichment_df.to_csv(outfile, index=False)
    
    return all_enrichment_df
        
        
                        
            
                    
            
        
#         for b, fold, t in zip(bs, folds, terms):

#             if term_column == 'GOs':
#                 name, definition, obsolete = get_GO_info(t)
#                 if obsolete:
#                     t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
#                 else:
#                     t = f'{name.capitalize()} ({t}): {definition}'

#             if term_column.split('_')[0] == 'KEGG':
#                 dfs = pd.read_html(f'https://www.genome.jp/entry/{t}')

#                 data = dfs[4]
#                 entry = data.loc[data[0] == 'Entry'][1].values[0]
#                 name = data.loc[data[0] == 'Name'][1].values[0]

#                 t = f'{t}: {name}'

In [18]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')

In [19]:
lldf_nn3 = pd.read_csv('../embedding/test_nn3_leiden_label_df_round_1.csv')

In [20]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]
for gene in gene_list_test:
    print((lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == gene])['leiden_label_rna_seq'].values[0])

4
24
10
24
10
10
24
10
10
24
24
24
10
10
10
10
16
24
10
24
24
10
10
10
16
10
40
24
5
24
24
24
41
23
24
24
18
10
24
24


In [21]:
lldf_nn3['leiden_label_rna_seq'].unique()

array([49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33,
       32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16,
       15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [22]:
# %pdb

In [23]:
enrich_full_nn3 = get_enrichment_df(lldf_nn3, 'rna_seq', complete_annotation, outfile='./test_nn3_full_enrichment.csv')

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:24<00:00,  2.03it/s]


In [24]:
enrich_full_nn3 = pd.read_csv('./test_nn3_full_enrichment.csv')

In [25]:
enrich_full_nn3.head(40)

,module,term,info,fold_change,bonferroni
0,0.0,Z,Cytoskeleton,4.247240,1.335030e-18
1,0.0,T,Signal transduction mechanisms,1.395865,2.957352e-02
2,0.0,S,Function unknown,1.234319,2.749970e-02
3,0.0,D,"Cell cycle control, cell division, chromosome ...",3.395502,4.816431e-10
4,0.0,GO:0005856,Cytoskeleton: A cellular structure that forms ...,3.945512,2.432438e-07
5,0.0,GO:0005929,Cilium: A specialized eukaryotic organelle tha...,5.765141,2.840020e-03
6,0.0,GO:0006996,Organelle organization: A process that is carr...,2.452899,3.167779e-04
7,0.0,GO:0016043,Cellular component organization: A process tha...,2.048829,6.066976e-03
8,0.0,GO:0044430,Obsolete cytoskeletal part: OBSOLETE. Any cons...,4.027088,1.245529e-06
9,0.0,GO:0071840,Cellular component organization or biogenesis:...,2.014741,7.663285e-03


In [26]:
for term in enrich_full_nn3['term'].unique():
    if 'ko:' not in term and 'GO:' not in term and '.' not in term:
        print(term)

Z
T
S
D
Q
P
G
J
K
L
I
C
E
B
O
U
A
Y


In [27]:
print(max(enrich_full_nn3['module'].values))
print(min(enrich_full_nn3['module'].values))

not_enriched_mod_list = []
enriched_mod_list = []

for num in range(int(max(enrich_full_nn3['module'].values)) + 1):
    if num not in enrich_full_nn3['module'].values:
        not_enriched_mod_list.append(num)
    if num in enrich_full_nn3['module'].values:
        enriched_mod_list.append(num)

print(len(not_enriched_mod_list) + len(enriched_mod_list))
print(max(not_enriched_mod_list + enriched_mod_list))
print()

print(set(range(int(max(enrich_full_nn3['module'].values)) + 1)) - set(enrich_full_nn3['module'].values))
print(len(set(enrich_full_nn3['module'].values)))
print(len(enriched_mod_list))
print(len(not_enriched_mod_list))

48.0
0.0
49
48

{1, 34, 35, 4, 5, 38, 40, 41, 10, 11, 14, 46, 20, 25, 26}
34
34
15


In [28]:
print(not_enriched_mod_list)

[1, 4, 5, 10, 11, 14, 20, 25, 26, 34, 35, 38, 40, 41, 46]


In [29]:
enrich_full_nn3.loc[enrich_full_nn3['module']==72]

,module,term,info,fold_change,bonferroni


In [30]:
enrich_full_nn3['module'].unique()

array([ 0.,  2.,  3.,  6.,  7.,  8.,  9., 12., 13., 15., 16., 17., 18.,
       19., 21., 22., 23., 24., 27., 28., 29., 30., 31., 32., 33., 36.,
       37., 39., 42., 43., 44., 45., 47., 48.])

In [31]:
lldf_nn3['leiden_label_rna_seq'].unique()

array([49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33,
       32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16,
       15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [32]:
lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == 'TTHERM_01386050']

,TTHERM_ID,leiden_label_rna_seq
5566,TTHERM_01386050,27


In [33]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]
for gene in gene_list_test:
    print((lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == gene])['leiden_label_rna_seq'].values[0])

4
24
10
24
10
10
24
10
10
24
24
24
10
10
10
10
16
24
10
24
24
10
10
10
16
10
40
24
5
24
24
24
41
23
24
24
18
10
24
24


In [34]:
len(set(enrich_full_nn3['module'].values))

34

end here. Return to embedding notebook!